In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset

ds = load_dataset("SHASWATSINGH3101/Synthetic_letter_dataset_v.3")

In [ ]:
ds

In [ ]:
!pip install  bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl

In [ ]:
import os
# disable Weights and Biases
os.environ['WANDB_DISABLED']="true"

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np

In [ ]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )


In [ ]:
model_name='Qwen/Qwen2-0.5B-Instruct'
original_model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
print_gpu_utilization()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(base_model, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=100, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.1,num_beams=1,top_p=0.95,).to('cpu')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

In [ ]:


# Split the dataset into training and validation sets
train_test_split = ds['train'].train_test_split(test_size=0.1)  # 10% for validation

# Accessing the splits
training_dataset = train_test_split['train']
val_dataset = train_test_split['test']

# Display the sizes of the splits
print(f"Training dataset size: {len(training_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

In [ ]:
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

index = 10

prompt = val_dataset[index]['prompt']
response = val_dataset[index]['output']

formatted_prompt = f"Instruct: Follow the prompt.\n{prompt}\nOutput:\n"
res = gen(original_model,formatted_prompt,100,)
#print(res[0])
output = res[0].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'DATASET:\n{response}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

In [ ]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('prompt', 'output').
    Then concatenate them using two newline characters.
    :param sample: Sample dictionary
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Follow the prompt."
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"
    
    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['prompt']}" if sample.get("prompt") else None
    response = f"{RESPONSE_KEY}\n{sample['output']}" if sample.get("output") else None
    end = f"{END_KEY}"
    
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

In [ ]:
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

In [ ]:
from functools import partial

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """
    
    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)#, batched=True)
    
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    
    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [ ]:
print_gpu_utilization()

In [ ]:
max_length = get_max_length(base_model)
print(max_length)

training_data = preprocess_dataset(tokenizer, max_length,seed, training_dataset)
eval_data = preprocess_dataset(tokenizer, max_length,seed, val_dataset)

In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {training_data.shape}")
print(f"Validation: {eval_data.shape}")
print(training_data)

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

In [ ]:
print(original_model)

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Define the LoRA configuration
config = LoraConfig(
    r=32,  # Rank
    lora_alpha=32,
    target_modules=[
        'self_attn.q_proj',  # Attention query projection
        'self_attn.k_proj',  # Attention key projection
        'self_attn.v_proj',  # Attention value projection
        'self_attn.o_proj'   # Attention output projection
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional dropout for LoRA
    task_type="CAUSAL_LM",  # Task type for causal language modeling
)

# 1 - Enable gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

# 2 - Prepare the model for k-bit training using PEFT
original_model = prepare_model_for_kbit_training(original_model)

# 3 - Get the PEFT model with the specified LoRA configuration
peft_model = get_peft_model(original_model, config)

# Output the PEFT model to verify
print(peft_model)

In [ ]:
print(print_number_of_trainable_model_parameters(peft_model))

In [ ]:
output_dir = './peft-dialogue-summary-training/final-checkpoint'
import transformers

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=2000,
    learning_rate=2.5e-4,
    optim="paged_adamw_8bit",
    logging_steps=50,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=500,
    evaluation_strategy="steps",
    eval_steps=150,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=training_data,
    eval_dataset=eval_data,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
peft_training_args.device

In [ ]:
print_gpu_utilization()

In [ ]:
peft_trainer.train()

In [ ]:
print_gpu_utilization()

In [ ]:
# del original_model
# del peft_trainer
# torch.cuda.empty_cache()

In [ ]:
print_gpu_utilization()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "Qwen/Qwen2-0.5B-Instruct"
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, 
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/kaggle/working/peft-dialogue-summary-training/final-checkpoint/checkpoint-1000",torch_dtype=torch.float16,is_trainable=False)

In [ ]:
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

index = 1
prompt = val_dataset[index]['prompt']
response = val_dataset[index]['output']

in_data = f"Instruct: {prompt}\n{prompt}\nOutput:\n"

peft_model_res = gen(ft_model,in_data,259,)
peft_model_output = peft_model_res[0].split('Output:\n')[1]
#print(peft_model_output)
prefix, success, result = peft_model_output.partition('#End')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
# print(dash_line)
# print(f'BASELINE HUMAN SUMMARY:\n{response}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

In [ ]:
original_model = AutoModelForCausalLM.from_pretrained(base_model_id, 
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

In [ ]:
import pandas as pd

prompt = val_dataset[0:10]['prompt']
response = val_dataset[0:10]['output']


original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(prompt):
    human_baseline_text_output = response[idx]
    prompt = f"Instruct: Summarize the following conversation.\n{dialogue}\nOutput:\n"
    
    original_model_res = gen(original_model,prompt,100,)
    original_model_text_output = original_model_res[0].split('Output:\n')[1]
    
    peft_model_res = gen(ft_model,prompt,100,)
    peft_model_output = peft_model_res[0].split('Output:\n')[1]
    #print(peft_model_output)
    peft_model_text_output, success, result = peft_model_output.partition('#End')
    

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(response, original_model_summaries, peft_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['response', 'original_model_summaries', 'peft_model_summaries'])
df

In [ ]:
!pip install rouge_score

In [ ]:
import evaluate

rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=response[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=response[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

In [ ]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

In [ ]:
peft_model.save_pretrained("SHASWATSINGH3101/Qwen2-0.5B-Instruct_lora")

In [ ]:
tokenizer.save_pretrained("SHASWATSINGH3101/Qwen2-0.5B-Instruct_lora")

In [ ]:
ls SHASWATSINGH3101/Qwen2-0.5B-Instruct_lora

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
repo_id = "SHASWATSINGH3101/Qwen2-0.5B-Instruct_lora"

# Create the repository
api.create_repo(repo_id=repo_id, repo_type="model", private=False)  # Set private=True if you want it to be private

In [ ]:
api.upload_folder(
    folder_path="SHASWATSINGH3101/Qwen2-0.5B-Instruct_lora",  # Path to your model directory
    repo_id=repo_id,      # Repository ID on Hugging Face Hub
    repo_type="model",    # Specify the type of repo
    commit_message="Add PEFT model and tokenizer files",  # Commit message
)

# Adaptor loading :-

In [ ]:
!pip install  bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl

In [ ]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM

# Define the pre-trained model name
model_name = 'Qwen/Qwen2-0.5B-Instruct'

# Load tokenizer and base model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

# Specify the path to save adapters
adapters_path = 'SHASWATSINGH3101/Qwen2-0.5B-Instruct_lora'

# Initialize PeftModel with base model and adapters path
model = PeftModel.from_pretrained(base_model, adapters_path)

# Merge the trainer adapter with the base model and unload the adapter
model = model.merge_and_unload()

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_name='Qwen/Qwen2-0.5B-Instruct'
original_model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_name, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=100, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks, max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.1,num_beams=1,top_p=0.95,)
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

In [ ]:
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

index = 1
prompt =  "Write a letter informing someone of potential legal action due to a dispute or violation."
# response = 

in_data = f"Instruct: {prompt}\n{prompt}\nOutput:\n"

peft_model_res = gen(model,in_data,259,)
peft_model_output = peft_model_res[0].split('Output:\n')[1]
#print(peft_model_output)
prefix, success, result = peft_model_output.partition('#End')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
# print(dash_line)
# print(f'BASELINE HUMAN SUMMARY:\n{response}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

In [ ]:
base_model


In [ ]:
model.save_pretrained("SHASWATSINGH3101/Qwen2-0.5B-Instruct_lora_merge")

In [ ]:
tokenizer.save_pretrained("SHASWATSINGH3101/Qwen2-0.5B-Instruct_lora_merge")

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
repo_id = "SHASWATSINGH3101/Qwen2-0.5B-Instruct_lora_merge"

# Create the repository
api.create_repo(repo_id=repo_id, repo_type="model", private=False)

In [ ]:
api.upload_folder(
    folder_path="SHASWATSINGH3101/Qwen2-0.5B-Instruct_lora_merge",  # Path to your model directory
    repo_id=repo_id,      # Repository ID on Hugging Face Hub
    repo_type="model",    # Specify the type of repo
    commit_message="Add PEFT model and tokenizer files",  # Commit message
)